# Activity Coefficients in Electrolyte Solutions
_Mikael Lund_

In [1]:
%matplotlib inline
from __future__ import print_function
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import mdtraj as md
from math import sqrt, pi, exp
import os.path, os, sys, json, filecmp, copy
plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir
print(workdir)

/Users/mikael/github/faunus-notebooks/activity-coefficients


### Download and compile the MC software

In [26]:
%%bash -s "$workdir"
%cd -q $1

echo 'fau_example(excess "./" excess.cpp)' > mc/CMakeLists.txt

# the following lines are for compilation on LUNARC
#module add GCC/5.4.0-2.26
#module load CMake/3.5.2
#export CXX=/sw/easybuild/software/Core/GCCcore/5.4.0/bin/g++
#export CC=/sw/easybuild/software/Core/GCCcore/5.4.0/bin/gcc

if [ ! -d "faunus/" ]; then
    git clone https://github.com/mlund/faunus.git
    cd faunus
    #git checkout a2220dc8f98606d2ceb42f14c6a08ec0723d8787
else
    cd faunus
fi

cmake . -DCMAKE_BUILD_TYPE=RelWithDebInfo -DENABLE_APPROXMATH=on -DMYPLAYGROUND=$1/mc &>/dev/null
make excess -j4
%cd $1

[ 36%] Built target xdrfile
[ 81%] Built target libfaunus
[100%] Built target excess


bash: line 1: fg: no job control
bash: line 21: fg: no job control


### Simulation setup
Wall time, number of cores etc. Currently for the slurm system.

In [8]:
%%writefile submit.sh
#!/bin/bash
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -t 03:00:00
../mc/excess > out

Overwriting submit.sh


In [19]:
salt_range = [0.1] #, 0.02, 0.04, 0.08, 0.16]

In [51]:
%cd -q $workdir

def mkinput():
    js = {
            "moleculelist": {
                "salt": { "Ninit": 20, "atomic": True, "atoms": "Na Cl" }
            }, 
            "energy": {
                "nonbonded": { "coulomb": { "epsr": 80 } }
            }, 

            "moves": {
                "atomtranslate": { "salt": { "permol": True, "prob": 0.01 } }, 
                "atomgc": { "molecule": "salt" }
            }, 

            "system": {
                "mcloop"      : { "macro": 10, "micro": micro }, 
                "cuboid"      : { "len": 60 },
                "coulomb"     : { "epsr": 80 },
                "temperature" : 298.15
            }, 

            "atomlist": {
                "Na": { "q":  1.0, "r": 2.3, "eps":0.01, "dp": 50, "activity": salt }, 
                "Cl": { "q": -1.0, "r": 2.3, "eps":0.01, "dp": 50, "activity": salt }
            }
    }

    with open('excess.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))

# flow control:
equilibration = True   # if true, delete state file and start over
production    = True   # if true, start from present state file
slurm         = False  # if true, submit production runs to slurm cluster
override      = True   # if true, override existing files

for salt in salt_range:      # loop over salt concentrations (activities)

    # create/goto right directory
    pfx='I'+str(salt)
    if not os.path.isdir(pfx):
        %mkdir -p $pfx
    else:
        if override==False:
            break
    %cd $pfx

    # equilibration run
    if equilibration:
        !rm -fR state
        micro=1000
        mkinput()
        !../mc/excess > eq 2>&1
        !rm -fR analysis_out.json

    # production run
    if production:
        micro=10000
        mkinput()
        if slurm:
            !sbatch ../submit.sh > /dev/null
        else:
            !../mc/excess > out 2>&1

    %cd -q ..

print('done.')

/Users/mikael/github/faunus-notebooks/activity-coefficients/I0.1
done.


In [72]:
%cd -q $workdir
import json

picklefile = 'data.p' 

if os.path.isfile( picklefile ):        # if pickle file exists, use that
    print('loading from saved pickle')
    sets = pd.read_pickle( picklefile )
else:
    print('loading from MC raw data')   # otherwise extract from MC output

    data = {}
    for salt in salt_range:
        pfx='I'+str(salt)
        d = {}
        if os.path.isdir(pfx):
            %cd -q $pfx            
            results = json.load( open('move_out.json') )
            j  = results['moves']['moves/atomgc']['atoms']
            %cd -q ..

            data[str(salt)] = pd.Series(j)
    data = pd.Series(data)
    data.to_pickle( picklefile )

loading from saved pickle


In [73]:
f = plt.figure(figsize=(10,5))

for salt, d in data.iteritems():
    for ion, avg in d.iteritems():
        avg = pd.Series(avg)
        print(ion, avg.molarity, avg.gamma)
    #plt.plot(d.pH, 0.5*np.array(d.C), label=r'$c_s=$'+salt+' M', lw=3, color=colors[salt])

#plt.xlabel('pH')
#plt.ylabel(r'Charge, $\langle Z \rangle$')
#plt.legend(loc=0, frameon=False)
#plt.ylim((-50, 100))
#plt.xlim((3,11))
#plt.minorticks_on()

#f.tight_layout()

#plt.savefig('dimer-vs-monomer.pdf', bbox_inches='tight')

Cl 0.130271945436 0.767624983764
Na 0.130271945436 0.767624983764
